In [56]:
### Library Imports
import pandas as pd
import yfinance as yf
import numpy as np
from scipy import stats
import datetime
import matplotlib.pyplot as plt


In [57]:
### Function to Import Stock Data
def import_stock_data(tickers, start_date, end_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date, end_date)['Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date, end_date)['Close']
    
    # Reset index to include the Date as a column
    data = data.reset_index()

    return data

tickers = ['AAPL', 'MSFT']
start_date = '2025-01-01'
end_date = '2025-06-01'
stock_data = import_stock_data(tickers, start_date, end_date)
print(stock_data.head())


/var/folders/n5/bknqp6qs37l5l4bwv9wv3bs40000gn/T/ipykernel_53809/17277343.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data[tickers] = yf.download(tickers, start_date, end_date)['Close']
[*********************100%***********************]  2 of 2 completed

        Date        AAPL        MSFT
0 2025-01-02  243.263199  416.976868
1 2025-01-03  242.774368  421.728607
2 2025-01-06  244.410416  426.211365
3 2025-01-07  241.627136  420.752350
4 2025-01-08  242.115952  422.933960


In [58]:
### Compute the Percentage Change
def pct_change(tickers, df):
    # Add col with daily pct change for each ticker
    for t in tickers:
        df['PCT ' + t] = df[t].pct_change()
    
    # Drop null vals
    df.dropna(subset=['PCT ' + t for t in tickers], inplace=True)

    return df

### Function Call to Compute the Percentage Change
stock_data = pct_change(tickers, stock_data)
print(stock_data.head())


        Date        AAPL        MSFT  PCT AAPL  PCT MSFT
1 2025-01-03  242.774368  421.728607 -0.002009  0.011396
2 2025-01-06  244.410416  426.211365  0.006739  0.010629
3 2025-01-07  241.627136  420.752350 -0.011388 -0.012808
4 2025-01-08  242.115952  422.933960  0.002023  0.005185
5 2025-01-10  236.280045  417.345459 -0.024104 -0.013214


In [59]:
### Generate Buy Signals
''' 
Generate a "BUY" signal if the stock loses more than 5% of its value in a single day
'''
def generate_signals(df, drop_threshold):
    """
    Generate 'BUY' signals if a stock drops more than the threshold in a day.
    """
    for t in tickers:
        signal_col = 'Signal ' + t # Fix this logic - don't like col title vars for strings
        pct_col = 'PCT ' + t
        df[signal_col] = (df[pct_col] < drop_threshold).astype(int)
        # Test to see the number of generated signals
        print(f"{signal_col}: {df[signal_col].sum()} buy signals generated")

    return df

### Function Retun to Generate Buy Signals
stock_data = generate_signals(stock_data, drop_threshold = -0.02)
print(stock_data.head())


Signal AAPL: 20 buy signals generated
Signal MSFT: 10 buy signals generated
        Date        AAPL        MSFT  PCT AAPL  PCT MSFT  Signal AAPL  \
1 2025-01-03  242.774368  421.728607 -0.002009  0.011396            0   
2 2025-01-06  244.410416  426.211365  0.006739  0.010629            0   
3 2025-01-07  241.627136  420.752350 -0.011388 -0.012808            0   
4 2025-01-08  242.115952  422.933960  0.002023  0.005185            0   
5 2025-01-10  236.280045  417.345459 -0.024104 -0.013214            1   

   Signal MSFT  
1            0  
2            0  
3            0  
4            0  
5            0  


In [ ]:
### Trade Simulation Function
''' 
Enter trade the day after the signal and exit after a fixed holding period (ex. 5 days)
'''
def trade_sim(df, holding_days):
    ### Empty List to Store Trades
    trades = []

    # Iterate over the df, stopping with the number of holding days left (avoids out of bounds error) 
    for i in range(len(df) - holding_days):
        # Identify the Buy Signals - Locate rows with a "1" indicator
        if df.iloc[i]['Signal AAPL'] == 1:
            # Set Entry/Exit Date - For each signal date (i), set the entry date to be the next trading day (i + 1)
            entry_idx = i + 1
            exit_idx = i + holding_days

            entry_date = df.index[entry_idx]
            exit_date = df.index[exit_idx]

            # Record Entry and Exit Prices
            entry_price = df.iloc[entry_idx]['AAPL']
            exit_price = df.iloc[exit_idx]['AAPL']

            # Compute the Return for the Asset
            returns = (exit_price / entry_price) - 1

            # Save trade to list
            trades.append({
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Entry Price': entry_price,
                'Exit Price': exit_price,
                'Return': returns
            })

    return pd.DataFrame(trades, columns = ['Entry Date', 'Exit Date', 'Entry Price', 'Exit Price', 'Return'])

trades = trade_sim(stock_data, holding_days = 5)
print(trades)


SyntaxError: invalid syntax (3001254060.py, line 38)